In [1]:
import os
from typing import Optional, List, Dict, Tuple, Union
from datetime import datetime

import pandas as pd
from datasets import load_dataset

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from dataset.radiolung import RadioLungDataset



c:\Users\xisca\anaconda\envs\torchmmm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path_list = ["11112024_BDMetaData_CanRuti.xlsx", "11112024_BDMetaData_santpau.xlsx", "28102024_BDMetaData_delmar.xlsx", "28102024_BDMetaData_mutuaterrassa.xlsx"]

data = RadioLungDataset(path_df=path_list)

#data.map_dataset()
df = data.get_df()
print(df)

     patient_id       hospital  type    sex   BMI                   education  \
0             1        canruti     1  Woman  33.0   Post high school training   
1             8        canruti     1    Man  33.0           High scrool grade   
2             9        CanRuti     1    Man  27.0           High scrool grade   
3            10        CanRuti     1    Man  31.2   Post high school training   
4            11        CanRuti     1    Man  23.5   Post high school training   
..          ...            ...   ...    ...   ...                         ...   
183         201  mutuaterrassa     1  Woman  26.9           High scrool grade   
184         205  mutuaterrassa     0    Man  30.0           High scrool grade   
185         206  mutuaterrassa     1    Man  29.4       Less than high school   
186         207  mutuaterrassa     1    Man  30.2       Less than high school   
200         222  mutuaterrassa     1  Woman  20.9       Less than high school   

     air_pollution  smoking

In [5]:
df=data

In [ ]:
import pandas as pd
from datetime import datetime
from datasets import load_dataset

def map_dataset(df, label_key="Diabetes_binary"):

    
    df.dropna(inplace=True)
    
    birthdate_cols = [col for col in df.columns if col.lower() in ['birthdate', 'birth_date']]
    if birthdate_cols:
        birthdate_col = birthdate_cols[0]  # Use the first found birthdate column
        df[birthdate_col] = pd.to_datetime(df[birthdate_col])
        df["age"] = datetime.today().year - df[birthdate_col].dt.year
        df.drop(columns=[birthdate_col], inplace=True)
    
    if 'ID' in df.columns:
        df.rename(columns={'ID': 'patient_id'}, inplace=True)
    
    if label_key in df.columns:
        values = df[label_key].unique()
        print(f"Unique values in {label_key}: {values}")
        
        if len(values) == 2:
            values = sorted(values)
            mapping = {values[0]: 0, values[1]: 1}
            print(f"Mapping: {mapping}")
            df[label_key] = df[label_key].map(mapping)
        
    
    print(f"Dataset shape after preprocessing: {df.shape}")
    return df

ds = load_dataset("Bena345/cdc-diabetes-health-indicators")
df = ds['train'].to_pandas()

print("Original dataset info:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

df_processed = map_dataset(df.copy())  
df = df_processed

Original dataset info:
Shape: (194825, 23)
Columns: ['ID', 'BMI', 'PhysHlth', 'Age', 'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'DiffWalk', 'Sex', 'Education', 'Income', 'Diabetes_binary']

First few rows:
       ID   BMI  PhysHlth          Age HighBP HighChol CholCheck Smoker  \
0  114414  29.0       0.0     65 to 69      0        1         1      0   
1  168896  32.0       0.0  80 or older      1        1         1      0   
2   68354  25.0       5.0     65 to 69      1        0         1      1   
3  121194  24.0       0.0  80 or older      1        0         1      0   
4  141150  31.0       0.0     25 to 29      0        0         1      0   

  Stroke HeartDiseaseorAttack  ... HvyAlcoholConsump AnyHealthcare  \
0      0                    0  ...                 0             1   
1      0                    0  ...                 0  

In [ ]:
def entrenar_random_forest(df, target_column='indice'):
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # categoric --> numeric
    for col in X.select_dtypes(include=['object', 'category']).columns:
        X[col] = LabelEncoder().fit_transform(X[col])

    if y.dtype == 'object' or y.dtype.name == 'category':
        y = LabelEncoder().fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = RandomForestClassifier(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    y_pred_proba0 = model.predict_proba(X_test)[:, 0]  # Probabilitats de la classe positiva
    y_pred_proba1 = model.predict_proba(X_test)[:, 1]  # Probabilitats de la classe negativa

    acc_test = accuracy_score(y_test, y_pred)
    auc_test0 = roc_auc_score(y_test, y_pred_proba0)
    auc_test1 = roc_auc_score(y_test, y_pred_proba1)

    test_report_dict = classification_report(y_test, y_pred, output_dict=True)
    recall0_test = test_report_dict['0']['recall']
    recall1_test = test_report_dict['1']['recall']
    
    auc0_test = auc_test0  
    auc1_test = auc_test1  
    
    print(f"\n=== RESULTATS TEST ===")
    print(f"AUC0: {auc0_test:.4f}")
    print(f"AUC1: {auc1_test:.4f}")
    print(f"Recall0: {recall0_test:.4f}")
    print(f"Recall1: {recall1_test:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    y_pred_train = model.predict(X_train)
    y_pred_proba_train0 = model.predict_proba(X_train)[:, 0]
    y_pred_proba_train1 = model.predict_proba(X_train)[:, 1]

    acc_train = accuracy_score(y_train, y_pred_train)
    auc_train0 = roc_auc_score(y_train, y_pred_proba_train0)
    auc_train1 = roc_auc_score(y_train, y_pred_proba_train1)

    train_report_dict = classification_report(y_train, y_pred_train, output_dict=True)
    recall0_train = train_report_dict['0']['recall']
    recall1_train = train_report_dict['1']['recall']
    
    auc0_train = auc_train0  
    auc1_train = auc_train1  
    
    print(f"\n=== RESULTATS TRAIN ===")
    print(f"AUC0: {auc0_train:.4f}")
    print(f"AUC1: {auc1_train:.4f}")
    print(f"Recall0: {recall0_train:.4f}")
    print(f"Recall1: {recall1_train:.4f}")
    print("Classification Report:")
    print(classification_report(y_train, y_pred_train))

    return {
        'train_report': classification_report(y_train, y_pred_train),
        'test_report': classification_report(y_test, y_pred),
        'train_auc0': auc0_train,
        'train_auc1': auc1_train,
        'test_auc0': auc0_test,
        'test_auc1': auc1_test,
        'train_recall0': recall0_train,
        'train_recall1': recall1_train,
        'test_recall0': recall0_test,
        'test_recall1': recall1_test,
        'train_accuracy': acc_train,
        'test_accuracy': acc_test
    }

In [8]:
entrenar_random_forest(df, 'type')


=== RESULTATS TEST ===
AUC0: 0.2946
AUC1: 0.7054
Recall0: 0.0000
Recall1: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.88      1.00      0.93        28

    accuracy                           0.88        32
   macro avg       0.44      0.50      0.47        32
weighted avg       0.77      0.88      0.82        32


=== RESULTATS TRAIN ===
AUC0: 0.0002
AUC1: 0.9998
Recall0: 1.0000
Recall1: 0.9896
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        31
           1       1.00      0.99      0.99        96

    accuracy                           0.99       127
   macro avg       0.98      0.99      0.99       127
weighted avg       0.99      0.99      0.99       127



c:\Users\xisca\anaconda\envs\torchmmm\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xisca\anaconda\envs\torchmmm\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xisca\anaconda\envs\torchmmm\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c

{'train_report': '              precision    recall  f1-score   support\n\n           0       0.97      1.00      0.98        31\n           1       1.00      0.99      0.99        96\n\n    accuracy                           0.99       127\n   macro avg       0.98      0.99      0.99       127\nweighted avg       0.99      0.99      0.99       127\n',
 'test_report': '              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00         4\n           1       0.88      1.00      0.93        28\n\n    accuracy                           0.88        32\n   macro avg       0.44      0.50      0.47        32\nweighted avg       0.77      0.88      0.82        32\n',
 'train_auc0': np.float64(0.0001680107526881719),
 'train_auc1': np.float64(0.9998319892473119),
 'test_auc0': np.float64(0.29464285714285715),
 'test_auc1': np.float64(0.7053571428571428),
 'train_recall0': 1.0,
 'train_recall1': 0.9895833333333334,
 'test_recall0': 0.0,
 'test_recall1': 1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import pandas as pd

def entrenar_xgboost(df, target_column='indice'):
  
    X = df.drop(columns=[target_column]).copy()
    y = df[target_column].copy()
    
    print(f"Dataset shape: {X.shape}")
    print(f"Target distribution:\n{pd.Series(y).value_counts()}")
    
    label_encoders = {}  
    
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) > 0:
        print(f"Converting categorical columns: {list(categorical_cols)}")
        for col in categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            label_encoders[col] = le
    
    target_encoder = None
    if y.dtype == 'object' or pd.api.types.is_categorical_dtype(y):
        print("Converting target variable to numeric")
        target_encoder = LabelEncoder()
        y = target_encoder.fit_transform(y)
        print(f"Target classes: {target_encoder.classes_}")
    
    unique_classes = pd.Series(y).nunique()
    is_binary = unique_classes == 2
    
    print(f"Number of classes: {unique_classes}")
    print(f"Binary classification: {is_binary}")
    
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y)
    
    print(f"Training set size: {X_train.shape[0]}")
    print(f"Test set size: {X_test.shape[0]}")
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    print("Training model...")
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    acc_test = accuracy_score(y_test, y_pred)
    
    auc_test = None
    if is_binary:
        y_pred_proba = model.predict_proba(X_test)[:, 0] 
        auc_test = roc_auc_score(y_test, y_pred_proba)
    
    print(f"\n=== RESULTATS TEST ===")
    print(f"Accuracy: {acc_test:.4f}")
    if auc_test is not None:
        print(f"AUC: {auc_test:.4f}")
    else:
        print("AUC: N/A (multiclass classification)")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # TRAIN 
    y_pred_train = model.predict(X_train)
    acc_train = accuracy_score(y_train, y_pred_train)
    
    auc_train = None
    if is_binary:
        y_pred_proba_train = model.predict_proba(X_train)[:, 0]
        auc_train = roc_auc_score(y_train, y_pred_proba_train)
    
    print(f"\n=== RESULTATS TRAIN ===")
    print(f"Accuracy: {acc_train:.4f}")
    if auc_train is not None:
        print(f"AUC: {auc_train:.4f}")
    else:
        print("AUC: N/A (multiclass classification)")
    print("Classification Report:")
    print(classification_report(y_train, y_pred_train))
    
    if hasattr(model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\nTop 10 Most Important Features:")
        print(feature_importance.head(10))
    
    return {
        'model': model,
        'train_report': classification_report(y_train, y_pred_train, output_dict=True),
        'test_report': classification_report(y_test, y_pred, output_dict=True),
        'train_auc': auc_train,
        'test_auc': auc_test,
        'train_accuracy': acc_train,
        'test_accuracy': acc_test,
        'feature_importance': feature_importance if hasattr(model, 'feature_importances_') else None,
        'label_encoders': label_encoders,
        'target_encoder': target_encoder,
        'is_binary': is_binary
    }

sults = entrenar_xgboost(df, 'type')


Dataset shape: (159, 15)
Target distribution:
type
1    124
0     35
Name: count, dtype: int64
Converting categorical columns: ['hospital', 'sex', 'education', 'smoking', 'fh_cancer']
Number of classes: 2
Binary classification: True
Training set size: 127
Test set size: 32
Training model...

=== RESULTATS TEST ===
Accuracy: 0.7812
AUC: 0.2257
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.78      1.00      0.88        25

    accuracy                           0.78        32
   macro avg       0.39      0.50      0.44        32
weighted avg       0.61      0.78      0.69        32


=== RESULTATS TRAIN ===
Accuracy: 0.9921
AUC: 0.0002
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        28
           1       0.99      1.00      0.99        99

    accuracy                           0.99       127
   macro a

C:\Users\xisca\AppData\Local\Temp\ipykernel_14248\3793149781.py:32: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if y.dtype == 'object' or pd.api.types.is_categorical_dtype(y):
c:\Users\xisca\anaconda\envs\torchmmm\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xisca\anaconda\envs\torchmmm\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\xisca\anaconda\envs\torchmmm\Lib\site-packag